<a href="https://colab.research.google.com/github/jeremi0214/DATA425_Assignment2/blob/main/LR_mobilenet_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Mobilenet v2 as fixed feature extractor on Oxford 102 Flowers or Caltech-101

import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow_datasets as tfds
from PIL import Image

In [5]:
# --- Configuration ---
use_caltech = True  # Set False for Oxford 102 Flowers
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# --- Transformations ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [23]:
# --- Load Dataset from tensorflow_datasets ---
dataset_name = 'oxford_flowers102' # Oxford flowers dataset
dataset, info = tfds.load(dataset_name, with_info=True, as_supervised=True)
train_data = dataset['train']
val_data = dataset['validation'] if 'validation' in dataset else dataset['test']

In [24]:
# --- Preprocess Dataset ---
def preprocess_tfds(data):
    data_list = []
    for img, label in tfds.as_numpy(data):
        img = Image.fromarray(img)
        img = transform(img)
        data_list.append((img, label))
    imgs = torch.stack([item[0] for item in data_list])
    labels = torch.tensor([item[1] for item in data_list])
    return DataLoader(TensorDataset(imgs, labels), batch_size=batch_size, shuffle=True)

train_loader = preprocess_tfds(train_data)
test_loader = preprocess_tfds(val_data)

In [25]:
# --- Load Pretrained MobileNet v2 ---
model = models.mobilenet_v2(pretrained=True)
model.eval()
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [26]:
# --- Feature Extraction Function ---
def extract_features(model, dataloader):
    features, labels = [], []
    with torch.no_grad():
        for imgs, lbls in dataloader:
            imgs = imgs.to(device)
            x = model.features(imgs)  # Extract features
            x = x.mean([2, 3])  # Global average pooling
            features.append(x.cpu().numpy())
            labels.append(lbls.numpy())
    return np.concatenate(features), np.concatenate(labels)

In [27]:
# --- Extract Features ---
X_train, y_train = extract_features(model, train_loader)
X_test, y_test = extract_features(model, test_loader)

In [28]:
# --- Train Logistic Regression ---
clf = LogisticRegression(solver='lbfgs', penalty='l2', max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [29]:
# --- Evaluate ---
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Top-1 Accuracy: {acc:.4f}")

Top-1 Accuracy: 0.8794
